In [37]:
'''
환경 : 쥬피터 노트북
0. https://pypi.org/project/googletrans/
번역 api - 각 단계 진행 시 해당 string 번역하여 값 반환
하이퍼링크 제거 메서드 생성

1. 테이블 정의 - 레시피, 요리정보, 재료, 요리단계

레시피 - Recipe
1. id - db 자동배정
2. foodName : title
3. foodImg : image
4. tempId : spoonacular api recipeId

요리정보 - FoodInformation
1. id - db 자동 배정
2. text - summary
3. cookingTime - readyInMinutes
4. serving - servings

재료 - Ingredients 항목에서 가져오기
1. id - db 자동배정
2. ingredientName : ingredientName
3. amount : amount

요리 단계 - CookStep : analyzedInstruction 항목에서 가져오기
1. id - db 자동
2. stepNumber - number
3. text - step
4. stepImg - null로 통일

2. docker - mysql
docker volume 설정으로 레시피 및 db 저장 폴더 지정
파이썬 mysql 연결 flask x

3. 레시피 아이디 들고와서 레시피 객체 생성 후 db 저장
4. 레시피 아이디를 리스트로 받아오기
5. 받아온 아이디를 기반으로 단일 레시피 get 요청 진행
    5-1. 요리 정보 : 데이터 분리, body에서 summary 추출,이미지 링크 삭제, 그리고 매핑 및 객체 생성,
    5-2. 재료 : 테이블과 각 값 매핑해서 객체 생성,
    5-3. 요리 단계 : 테이블과 각 값 매핑해서 객체 생성
    각 순서마다 db에 상황마다 바로 전송
    
API key : 03351b92426b464499f9b05790d6894b
'''

'\n환경 : 쥬피터 노트북\n0. https://pypi.org/project/googletrans/\n번역 api - 각 단계 진행 시 해당 string 번역하여 값 반환\n하이퍼링크 제거 메서드 생성\n\n1. 테이블 정의 - 레시피, 요리정보, 재료, 요리단계\n\n레시피 - Recipe\n1. id - db 자동배정\n2. foodName : title\n3. foodImg : image\n4. tempId : spoonacular api recipeId\n\n요리정보 - FoodInformation\n1. id - db 자동 배정\n2. text - summary\n3. cookingTime - readyInMinutes\n4. serving - servings\n\n재료 - Ingredients 항목에서 가져오기\n1. id - db 자동배정\n2. ingredientName : ingredientName\n3. amount : amount\n\n요리 단계 - CookStep : analyzedInstruction 항목에서 가져오기\n1. id - db 자동\n2. stepNumber - number\n3. text - step\n4. stepImg - null로 통일\n\n2. docker - mysql\ndocker volume 설정으로 레시피 및 db 저장 폴더 지정\n파이썬 mysql 연결 flask x\n\n3. 레시피 아이디 들고와서 레시피 객체 생성 후 db 저장\n4. 레시피 아이디를 리스트로 받아오기\n5. 받아온 아이디를 기반으로 단일 레시피 get 요청 진행\n    5-1. 요리 정보 : 데이터 분리, body에서 summary 추출,이미지 링크 삭제, 그리고 매핑 및 객체 생성,\n    5-2. 재료 : 테이블과 각 값 매핑해서 객체 생성,\n    5-3. 요리 단계 : 테이블과 각 값 매핑해서 객체 생성\n    각 순서마다 db에 상황마다 바로 전송\n    \nAPI key : 03351b92426b464499f9b

In [38]:
! pip install sqlalchemy
! pip install googletrans==3.1.0a0

In [39]:
class Recipe:
    def __init__(self, recipe_id, food_name, food_img, temp_id):
        self.recipe_id = recipe_id  # DB에서 자동으로 할당됨
        self.food_name = food_name
        self.food_img = food_img
        self.temp_id = temp_id

    def updateFoodName(self, food_name):
        self.food_name = food_name


class FoodInformation:
    def __init__(self, recipe_id, text, cooking_time, serving):
        self.id = None  # DB에서 자동으로 할당됨
        self.recipe_id = recipe_id
        self.text = text
        self.cooking_time = cooking_time
        self.serving = serving

    def updateText(self, text):
        self.text = text


class Ingredient:
    def __init__(self, recipe_id, ingredient_name, amount):
        self.id = None  # DB에서 자동으로 할당됨
        self.recipe_id = recipe_id
        self.ingredient_name = ingredient_name
        self.amount = amount

    def updateIngredient(self, ingredient_name):
        self.ingredient_name = ingredient_name

class CookStep:
    def __init__(self, recipe_id, step_number, text, step_img):
        self.id = None  # DB에서 자동으로 할당됨
        self.recipe_id = recipe_id
        self.step_number = step_number
        self.text = text
        self.step_img = step_img

    def updateCookstep(self, text):
        self.text = text
       
       




In [40]:
! pip install PyMySQL;

In [41]:
from sqlalchemy.dialects.oracle import LONG
from sqlalchemy import Column, Integer, String, ForeignKey, DateTime, create_engine
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects import mysql


# 데이터베이스 연결 설정
db_name = "tastyhub"
db_host = "localhost"
db_username = "root"
db_password = "1234"

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}")

# 세션 생성
Session = sessionmaker(bind=engine)
session = Session()

# 기본 클래스 생성
Base = declarative_base()


# Recipe 클래스를 데이터베이스 테이블로 매핑
class TRecipe(Base):
    __tablename__ = 'recipes'

    recipe_id = Column(mysql.BIGINT, primary_key=True, autoincrement=True)
    food_name = Column(String(255))
    food_img = Column(String(255))
    # created_at = Column(DateTime)
    # modified_at = Column(DateTime)
    user_id = Column(Integer)

    food_information = relationship('TFoodInformation', back_populates='recipe')
    ingredient = relationship('TIngredient', back_populates='recipe')
    cook_steps = relationship('TCookStep', back_populates='recipe')

# FoodInformation 클래스를 데이터베이스 테이블로 매핑
class TFoodInformation(Base):
    __tablename__ = 'food_information'

    food_information_id = Column(mysql.BIGINT, primary_key=True, autoincrement=True)
    recipe_id = Column(mysql.BIGINT, ForeignKey('recipes.recipe_id'))
    text = Column(String(1000))
    cooking_time = Column(Integer)
    serving = Column(Integer)
    created_at = Column(DateTime)
    modified_at = Column(DateTime)

    recipe = relationship('TRecipe', back_populates='food_information')

# Ingredient 클래스를 데이터베이스 테이블로 매핑
class TIngredient(Base):
    __tablename__ = 'ingredient'

    ingredient_id = Column(mysql.BIGINT, primary_key=True, autoincrement=True)
    recipe_id = Column(mysql.BIGINT, ForeignKey('recipes.recipe_id'))
    ingredient_name = Column(String(255))
    amount = Column(String(100))
    created_at = Column(DateTime)
    modified_at = Column(DateTime)

    recipe = relationship('TRecipe', back_populates='ingredient')

# CookStep 클래스를 데이터베이스 테이블로 매핑
class TCookStep(Base):
    __tablename__ = 'cook_steps'
    __tablename__ = 'cook_steps'

    cookstep_id = Column(mysql.BIGINT, primary_key=True, autoincrement=True)
    
    recipe_id = Column(mysql.BIGINT, ForeignKey('recipes.recipe_id'))
    step_number = Column(Integer)
    text = Column(String(1000))
    step_img_url = Column(String(255))
    created_at = Column(DateTime)
    modified_at = Column(DateTime)

    recipe = relationship('TRecipe', back_populates='cook_steps')
# 테이블 생성
Base.metadata.create_all(engine)

# 세션 종료
session.close()

C:\Users\tpgus\AppData\Local\Temp\ipykernel_14228\607999336.py:22: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [42]:
# Recipe 객체를 TRecipe 객체로 변환하는 함수
def convert_to_trecipe(recipe):
    print(recipe.food_name)
    trecipe = TRecipe(
        # recipe_id=recipe.recipe_id,
        food_name=recipe.food_name,
        food_img=recipe.food_img,
        # created_at=DateTime.
        user_id=1
    )
    print(trecipe.food_name)
    return trecipe

# FoodInformation 객체를 TFoodInformation 객체로 변환하는 함수
def convert_to_tfoodinformation(food_information,trecipe):
    tfood_information = TFoodInformation(
        recipe_id=food_information.recipe_id,
        text=food_information.text,
        cooking_time=food_information.cooking_time,
        serving=food_information.serving,
        recipe = trecipe
    )
    return tfood_information

# Ingredient 객체를 TIngredient 객체로 변환하는 함수
def convert_to_tingredient(ingredient,trecipe):
    tingredient = TIngredient(
        recipe_id=ingredient.recipe_id,
        ingredient_name=ingredient.ingredient_name,
        amount=ingredient.amount,
        recipe = trecipe
    )
    return tingredient

# CookStep 객체를 TCookStep 객체로 변환하는 함수
def convert_to_tcookstep(cook_step,trecipe):
    tcook_step = TCookStep(
        recipe_id=cook_step.recipe_id,
        step_number=cook_step.step_number,
        text=cook_step.text,
        step_img_url=cook_step.step_img,
        recipe = trecipe
    )
    return tcook_step


In [43]:
import requests

all_search_url = 'https://api.spoonacular.com/recipes/complexSearch?number=&apiKey=03351b92426b464499f9b05790d6894b'
response = requests.get(all_search_url)

In [44]:
print(response.text)

{"results":[{"id":715415,"title":"Red Lentil Soup with Chicken and Turnips","image":"https://img.spoonacular.com/recipes/715415-312x231.jpg","imageType":"jpg"},{"id":716406,"title":"Asparagus and Pea Soup: Real Convenience Food","image":"https://img.spoonacular.com/recipes/716406-312x231.jpg","imageType":"jpg"},{"id":644387,"title":"Garlicky Kale","image":"https://img.spoonacular.com/recipes/644387-312x231.jpg","imageType":"jpg"},{"id":715446,"title":"Slow Cooker Beef Stew","image":"https://img.spoonacular.com/recipes/715446-312x231.jpg","imageType":"jpg"},{"id":782601,"title":"Red Kidney Bean Jambalaya","image":"https://img.spoonacular.com/recipes/782601-312x231.jpg","imageType":"jpg"},{"id":716426,"title":"Cauliflower, Brown Rice, and Vegetable Fried Rice","image":"https://img.spoonacular.com/recipes/716426-312x231.jpg","imageType":"jpg"},{"id":716004,"title":"Quinoa and Chickpea Salad with Sun-Dried Tomatoes and Dried Cherries","image":"https://img.spoonacular.com/recipes/716004-312

In [45]:
import json

In [46]:
data = json.loads(response.text)
list_id = 0
recipes = []
# Iterate over each item in the list of results
for result in data["results"]:
    # Access each attribute of the result
    print(type(result))
    recipe_id = list_id
    title = result["title"]
    image = result["image"]
    temp_id = result["id"]
    
    recipe = Recipe(recipe_id, title, image, temp_id)
    print(recipe)

    recipes.append(recipe)
    list_id += 1

print(recipes[0].food_name)  # Just for formatting

print(len(recipes))

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
Red Lentil Soup with Chicken and Turnips
10


In [47]:
foodInformations = []
ingredients = []
cookSteps = []
for obj in recipes:
    recipe_temp_id = obj.temp_id

    recipe_search_url = 'https://api.spoonacular.com/recipes/informationBulk?ids=' + str(
        recipe_temp_id) + '&includeNutrition=false&apiKey=03351b92426b464499f9b05790d6894b'
    response = requests.get(recipe_search_url)
    text = response.text
    data = json.loads(text)
    for foodInfo in data:
        text = foodInfo['summary']
        cookingTime = foodInfo['readyInMinutes']
        serving = foodInfo['servings']
        foodInformation = FoodInformation(obj.recipe_id,text,cookingTime,serving)
        foodInformations.append(foodInformation)

        for ing in foodInfo['extendedIngredients']:
            ingredient_name = ing['name']
            amount = ing['amount']
            ingredient = Ingredient(obj.recipe_id,ingredient_name, amount)
            ingredients.append(ingredient)

        for cks in foodInfo['analyzedInstructions']:
            ck = cks['steps']
            for c in ck:
                stepNumber = c['number']
                text = c['step']
                stepImg = "afterRefactor"
                cookStep = CookStep(obj.recipe_id,stepNumber,text,stepImg)
                cookSteps.append(cookStep)
    


In [48]:
for rec in recipes:
    id = rec.recipe_id
    print(id)
    print(rec.food_name)
    for fi in foodInformations:
        if id == fi.recipe_id:
            print(fi.text)
            print(fi.cooking_time)
            print(fi.serving)
            print()

    for ig in ingredients:
        if id == ig.recipe_id:
            print(ig.ingredient_name)
            print(ig.amount)
            print()

    for cs in cookSteps:
        if id == cs.recipe_id:
            print(cs.text)
            print(cs.step_number)
            print(cs.step_img)
            print()

    print("---------다음 레시피--------")

0
Red Lentil Soup with Chicken and Turnips
Red Lentil Soup with Chicken and Turnips might be a good recipe to expand your main course repertoire. This recipe serves 8 and costs $3.0 per serving. One serving contains <b>477 calories</b>, <b>27g of protein</b>, and <b>20g of fat</b>. It is brought to you by Pink When. 1866 people have tried and liked this recipe. It can be enjoyed any time, but it is especially good for <b>Autumn</b>. From preparation to the plate, this recipe takes approximately <b>55 minutes</b>. It is a good option if you're following a <b>gluten free and dairy free</b> diet. Head to the store and pick up salt and pepper, canned tomatoes, flat leaf parsley, and a few other things to make it today. Overall, this recipe earns a <b>spectacular spoonacular score of 99%</b>. If you like this recipe, you might also like recipes such as <a href="https://spoonacular.com/recipes/red-lentil-and-chicken-soup-682185">Red Lentil and Chicken Soup</a>, <a href="https://spoonacular.c

In [49]:
import re

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


In [50]:
for fi in foodInformations:
    text = fi.text
    text = remove_html_tags(text)
    fi.updateText(text)
    print(fi.text)

Red Lentil Soup with Chicken and Turnips might be a good recipe to expand your main course repertoire. This recipe serves 8 and costs $3.0 per serving. One serving contains 477 calories, 27g of protein, and 20g of fat. It is brought to you by Pink When. 1866 people have tried and liked this recipe. It can be enjoyed any time, but it is especially good for Autumn. From preparation to the plate, this recipe takes approximately 55 minutes. It is a good option if you're following a gluten free and dairy free diet. Head to the store and pick up salt and pepper, canned tomatoes, flat leaf parsley, and a few other things to make it today. Overall, this recipe earns a spectacular spoonacular score of 99%. If you like this recipe, you might also like recipes such as Red Lentil and Chicken Soup, Red Lentil and Chicken Soup, and Red-Lentil Soup.
Asparagus and Pea Soup: Real Convenience Food requires approximately 20 minutes from start to finish. Watching your figure? This gluten free, dairy free,

In [51]:
# 번역 메서드
from googletrans import Translator
#\ https://pypi.org/project/googletrans/

translator = Translator()

for rec in recipes:
    id = rec.recipe_id
    text = rec.food_name
    translated = translator.translate(text,src="en", dest="ko")

    rec.updateFoodName(str(translated.text))
    print(rec.food_name)
    for fi in foodInformations:
        if id == fi.recipe_id:
            fi.updateText(str(translator.translate(fi.text,dest="ko").text))


    for ig in ingredients:
        if id == ig.recipe_id:
            ig.updateIngredient(str(translator.translate(ig.ingredient_name, dest="ko").text))


    for cs in cookSteps:
        if id == cs.recipe_id:
            cs.updateCookstep(str(translator.translate(cs.text, dest="ko").text))




닭고기와 순무를 곁들인 빨간 렌즈콩 수프
아스파라거스와 완두콩 수프: 진짜 간편식
갈릭케일
슬로우 쿠커 쇠고기 스튜
붉은 강낭콩 잠발라야
콜리플라워, 현미, 야채 볶음밥
햇볕에 말린 토마토와 말린 체리를 곁들인 퀴노아와 병아리콩 샐러드
집에서 쉽게 만드는 쌀과 콩
올리브 오일과 로즈마리를 곁들인 토스카나 흰 콩 수프
바삭바삭한 브뤼셀 콩나물 반찬


In [52]:
for rec in recipes:
    id = rec.recipe_id
    print(id)
    print(rec.food_name)
    for fi in foodInformations:
        if id == fi.recipe_id:
            print(fi.text)
            print(fi.cooking_time)
            print(fi.serving)
            print()

    for ig in ingredients:
        if id == ig.recipe_id:
            print(ig.ingredient_name)
            print(ig.amount)
            print()

    for cs in cookSteps:
        if id == cs.recipe_id:
            print(cs.text)
            print(cs.step_number)
            print(cs.step_img)
            print()

    print("---------다음 레시피--------")


0
닭고기와 순무를 곁들인 빨간 렌즈콩 수프
닭고기와 순무를 곁들인 붉은 렌즈콩 수프는 메인 코스 레퍼토리를 확장하는 데 좋은 요리법이 될 수 있습니다. 이 레시피는 8인분이며 가격은 1인분당 $3.0입니다. 1회 제공량에는 477칼로리, 단백질 27g, 지방 20g이 들어 있습니다. Pink When에서 여러분께 전해 드립니다. 1866명이 이 레시피를 시도하고 좋아했습니다. 언제든지 즐길 수 있지만 특히 가을에 좋습니다. 준비부터 접시 준비까지 이 레시피는 약 55분 정도 소요됩니다. 글루텐 프리 및 유제품 프리 식단을 따르고 있다면 좋은 선택입니다. 가게에 가서 소금과 후추, 토마토 통조림, 평평한 잎 파슬리, 그리고 오늘 만들 수 있는 몇 가지 다른 것들을 사세요. 전반적으로 이 레시피는 99%라는 놀라운 점수를 얻었습니다. 이 레시피가 마음에 드신다면 빨간색 렌즈콩과 닭고기 수프, 빨간색 렌즈콩과 닭고기 수프, 빨간색 렌즈콩 수프 같은 레시피도 마음에 드실 겁니다.
55
8

추가토핑 : 아보카도
8.0

당근
3.0

셀러리 줄기
3.0

닭고기 가슴살
2.0

평평한 잎 파슬리
0.5

마늘
6.0

올리브유
2.0

통조림 토마토
28.0

렌틸 콩
2.0

소금과 후추
8.0

순무
1.0

야채 육수
8.0

양파
1.0

대형 더치 오븐이나 수프 냄비에 올리브 오일을 넣고 중간 불로 가열합니다.
1
afterRefactor

양파, 당근, 셀러리를 추가하고 가끔씩 저어주면서 부드러워질 때까지 8~10분간 조리합니다.
2
afterRefactor

마늘을 추가하고 추가로 2분 동안 또는 향이 날 때까지 조리합니다. 약간의 소금과 후추로 양념을 잘 하세요. 냄비에 토마토, 순무, 붉은 렌즈콩을 넣으세요. 섞어서 섞으세요. 야채 스톡을 섞고 스토브의 불을 높이 올리세요. 수프를 끓인 다음 약불로 줄여 끓입니다. 20분간 또는 순무가 부드러워지고 렌즈콩이 완전히 익을 때까지 끓입니다.
3
afterRefactor

닭 가슴살과 파슬리를 추가합니다.

In [53]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

# 데이터베이스 연결 설정
db_name = "tastyhub"
db_host = "localhost"
db_username = "root"
db_password = "1234"

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}")

# 세션 생성
Session = sessionmaker(bind=engine)
session = Session()



In [54]:
Base = declarative_base()
try:
    print("start")
    for rec in recipes:
        trecipe = convert_to_trecipe(rec)
        session.add(trecipe)
        session.commit()
    
    recipe_objects = session.query(TRecipe).all()
    print(type(recipe_objects))
    
    for ro in recipe_objects:
        for fi in foodInformations:  # your_objects에는 객체들의 리스트가 들어가야 합니다.
            print(fi.recipe_id)
            print(ro.recipe_id)
            if fi.recipe_id+1==ro.recipe_id:
                tfoodinformation = convert_to_tfoodinformation(fi,ro)
                print(tfoodinformation.recipe_id)
                session.add(tfoodinformation)

        for ing in ingredients:  # your_objects에는 객체들의 리스트가 들어가야 합니다.
            if ing.recipe_id+1 == ro.recipe_id:
                tingredient = convert_to_tingredient(ing,ro)
                session.add(tingredient)

        for cs in cookSteps:  # your_objects에는 객체들의 리스트가 들어가야 합니다.
            if cs.recipe_id+1 == ro.recipe_id:
                tcookstep = convert_to_tcookstep(cs,ro)
                session.add(tcookstep)
            
    session.commit()



except Exception as e:
    session.rollback()
    print("Error:", e)

# 세션 종료
session.close()

start
닭고기와 순무를 곁들인 빨간 렌즈콩 수프
닭고기와 순무를 곁들인 빨간 렌즈콩 수프


C:\Users\tpgus\AppData\Local\Temp\ipykernel_14228\2577517727.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


아스파라거스와 완두콩 수프: 진짜 간편식
아스파라거스와 완두콩 수프: 진짜 간편식
갈릭케일
갈릭케일
슬로우 쿠커 쇠고기 스튜
슬로우 쿠커 쇠고기 스튜
붉은 강낭콩 잠발라야
붉은 강낭콩 잠발라야
콜리플라워, 현미, 야채 볶음밥
콜리플라워, 현미, 야채 볶음밥
햇볕에 말린 토마토와 말린 체리를 곁들인 퀴노아와 병아리콩 샐러드
햇볕에 말린 토마토와 말린 체리를 곁들인 퀴노아와 병아리콩 샐러드
집에서 쉽게 만드는 쌀과 콩
집에서 쉽게 만드는 쌀과 콩
올리브 오일과 로즈마리를 곁들인 토스카나 흰 콩 수프
올리브 오일과 로즈마리를 곁들인 토스카나 흰 콩 수프
바삭바삭한 브뤼셀 콩나물 반찬
바삭바삭한 브뤼셀 콩나물 반찬
<class 'list'>
0
1
0
1
1
2
1
3
1
4
1
5
1
6
1
7
1
8
1
9
1
0
2
1
2
1
2
2
3
2
4
2
5
2
6
2
7
2
8
2
9
2
0
3
1
3
2
3
2
3
3
4
3
5
3
6
3
7
3
8
3
9
3
0
4
1
4
2
4
3
4
3
4
4
5
4
6
4
7
4
8
4
9
4
0
5
1
5
2
5
3
5
4
5
4
5
5
6
5
7
5
8
5
9
5
0
6
1
6
2
6
3
6
4
6
5
6
5
6
6
7
6
8
6
9
6
0
7
1
7
2
7
3
7
4
7
5
7
6
7
6
7
7
8
7
9
7
0
8
1
8
2
8
3
8
4
8
5
8
6
8
7
8
7
8
8
9
8
0
9
1
9
2
9
3
9
4
9
5
9
6
9
7
9
8
9
8
9
9
0
10
1
10
2
10
3
10
4
10
5
10
6
10
7
10
8
10
9
10
9
